In [1]:
from neo4j import GraphDatabase

In [2]:
uri="bolt://localhost:7687"
user="neo4j"
password="88888888"

driver = GraphDatabase.driver(uri, auth=(user, password))

• Ranking de notícias: este recurso permite que, dado um usuário específico, sejam
exibidas uma lista de notícias mais relevantes para ele, baseado em seus interesses.

In [54]:
@staticmethod
def recommendNews(tx, userID):
    result = tx.run("MATCH path = (u:User {id: $userID})-[read:READ]->(n:News)-[content:CONTENT]->(e:Entity)-[content2:CONTENT]->(n2:News) "
                    "WHERE n2.id <> n.id "
                    "WITH e, n2, rand() AS rnd "
                    "WITH e, n2, collect(rnd)[0..1] AS rnd " # collect(n2) AS nn, collect(n2.id) AS n2ic, collect(n2.title) AS n2tc, collect(n2.abstract) AS n2ac,
                    "RETURN e.label, n2.id, n2.title, n2.abstract ORDER BY rnd LIMIT 10", userID=userID)
    return list(result)

In [60]:
userID = "U169207"
with driver.session() as session:
    newsList = session.execute_read(recommendNews, userID)
    for news in newsList: 
        print(news.get("e.label") + " - " + news.get("n2.id") + " - " + news.get("n2.title") + "\n\n\t" + news.get("n2.abstract") + "\n\n\n") # news.data() news.get("n2c[0]")

Donald Trump - N11290 - Trump on impeachment probe: 'The Democrats are crazed and they're lunatics'v

	President Donald Trump gives remarks as he departs for his Keep America Great rally in Mississippi.



Donald Trump - N2038 - Giuliani: I won't comply with "illegitimate" inquiry

	President Trump's personal attorney Rudy Giuliani said he will not comply with a subpoena from House Democrats, calling the impeachment inquiry "illegitimate." Lawmakers are seeking documents related to Giuliani's contacts with Ukraine. Anita Kumar and Sarah Wire joined "Red & Blue" to discuss the latest on the inquiry and Syria.



Country Music Association Awards - N57634 - Women rule the night at the CMA Awards

	Women ruled the night Wednesday at the 53rd Country Music Association Awards, starting with Jennifer Nettles's feminist fashion statement on the red carpet; co-hosts Carrie Underwood, Reba McEntire, and Dolly Parton's sassy monologue; and, most thrillingly, an opening number starring a dozen fie

• Ranking de usuários: este recurso permite que, dado uma notícia específica, sejam
exibidos uma lista de usuários que podem ter interesse nessa notícia, baseado em
seus interesses conhecidos.

In [72]:
@staticmethod
def recommendUsers(tx, newsID):
    result = tx.run("MATCH path = (n:News{id: $newsID})-[content:CONTENT]-(e:Entity)-[content2:CONTENT]-(n2:News)-[read2:READ]-(u2:User) "
                    "WHERE n2.id <> n.id "
                    "WITH e, n2, u2, rand() AS rnd "
                    "WITH e, n2, u2, collect(rnd)[0..1] AS rnd "
                    "RETURN e.label, n2.title, u2.id AS user ORDER BY rnd LIMIT 100", newsID=newsID)
    return list(result)

In [73]:
newsID = "N107483"
with driver.session() as session:
    userList = session.execute_read(recommendUsers, newsID)
    for user in userList: 
        print(news.get("e.label") + " - " + news.get("u2.id") )

TypeError: can only concatenate str (not "NoneType") to str

• Adição de novo usuário no grafo: este recurso permite que um novo usuário
seja adicionado ao grafo de relações existentes, informando suas preferências por
notícias. A partir disso, ser ́a poss ́ıvel listar as not ́ıcias mais relevantes para ele.

• Adicão de nova notícia no grafo: este recurso permite que uma nova notícia seja
adicionada ao grafo de relacões existentes, para que sejam listados os usuários
potenciais que podem ter interesse nela.

In [ ]:
driver.close()